# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: Ayush Kamleshbhai Patel
#### Student ID: 3891013

Date: XXXX

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction
You should give a brief information of this assessment task here.

## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.models import FastText
from sklearn.decomposition import PCA
import gc
from sklearn.linear_model import LogisticRegression

# import umap.umap_ as umap



## Task 2. Generating Feature Representations for Job Advertisement Descriptions

#### Loading the data

In [2]:
with open('description.txt') as txtf:
    article_txts = txtf.read().splitlines() # reading a list of strings, each for a document/article
tk_description = [a.split(' ') for a in article_txts]

In [3]:
words = list(chain.from_iterable(tk_description))
# set of unique words
vocab = sorted(list(set(words)))
# total number of the vocabulary
len(vocab)


5168

In [4]:
jobs_data = pd.read_csv('jobs_data.csv')
webindex = jobs_data['Webindex']
# Define the mapping
category_to_int_mapping = {
    'Accounting_Finance': 0,
    'Engineering': 1,
    'Healthcare_Nursing': 2,
    'Sales': 3
}

# Replace the labels with their integer values
jobs_data['category_encoded'] = jobs_data['Category'].map(category_to_int_mapping)
jobs_data['category_encoded'] = jobs_data['category_encoded'].astype(int)
jobs_data.head(5)

,Webindex,Title,Company,Description,Tokenized Description,Tokenized Title,Tokenized Company,Category,category_encoded
0,68997528,Finance / Accounts Asst Bromley to ****k,First Recruitment Services,Accountant (partqualified) to **** p.a. South ...,"['accountant', 'partqualified', 'south', 'east...","['finance', 'accounts', 'asst', 'bromley', 'to...","['first', 'recruitment', 'services']",Accounting_Finance,0
1,68063513,Fund Accountant Hedge Fund,Austin Andrew Ltd,One of the leading Hedge Funds in London is cu...,"['hedge', 'funds', 'london', 'recruiting', 'fu...","['fund', 'accountant', 'hedge', 'fund']","['austin', 'andrew', 'ltd']",Accounting_Finance,0
2,68700336,Deputy Home Manager,Caritas,An exciting opportunity has arisen to join an ...,"['exciting', 'arisen', 'establish', 'provider'...","['deputy', 'home', 'manager']",['caritas'],Healthcare_Nursing,2
3,67996688,Brokers Wanted Imediate Start,OneTwoTrade,OneTwoTrade is expanding their Sales Team and ...,"['expanding', 'recruiting', 'junior', 'trainee...","['brokers', 'wanted', 'imediate', 'start']",['onetwotrade'],Accounting_Finance,0
4,71803987,RGN Nurses (Hospitals) Penarth,Swiis Healthcare,RGN Nurses (Hospitals) Immediate fulltime and ...,"['rgn', 'nurses', 'hospitals', 'fulltime', 'pa...","['rgn', 'nurses', 'hospitals', 'penarth']","['swiis', 'healthcare']",Healthcare_Nursing,2


### Bag Of Words Model

#### Creating Binary Vectors

In [5]:
# Code to perform the task...
joined_description = [' '.join(review) for review in tk_description]
bVectorizer = CountVectorizer(analyzer = "word",binary = True,vocabulary = vocab) # initialise the CountVectorizer
binary_features = bVectorizer.fit_transform(joined_description)
binary_features.shape

(776, 5168)

#### Creating Count Vectors

In [6]:
cVectorizer = CountVectorizer(analyzer = "word",vocabulary = vocab) # initialised the CountVectorizer
count_features = cVectorizer.fit_transform(joined_description)
count_features.shape

(776, 5168)

In [7]:
count_features_df = pd.DataFrame(count_features.toarray(), columns=cVectorizer.get_feature_names_out())
# print out samples
count_features_df.head(3)

,aap,aaron,aat,abb,abenefit,aberdeen,abi,abilities,abreast,abroad,...,years,yeovil,yn,york,yorkshire,youmust,young,younger,yrs,zest
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Saving it into the file

In [8]:
count_features = cVectorizer.fit_transform(joined_description).toarray()

def save_count_vector(count_features, webindex, filename):
    with open(filename, 'w') as f:
        for i in range(len(count_features)):
            f.write('#' + str(webindex[i]) + ',')
            for j in range(len(count_features[i])):
                if count_features[i][j] != 0:
                    f.write(str(j) + ':' + str(count_features[i][j]) + ',')
            f.write('\n')
    f.close()
    print('Count vector representation saved to ' + filename)


save_count_vector(count_features, webindex, 'count_vectors.txt')

Count vector representation saved to count_vectors.txt


#### Creating TF-IDF Vectors

In [9]:
tVectorizer = TfidfVectorizer(analyzer = "word",vocabulary = vocab) # initialised the TfidfVectorizer
tfidf_features = tVectorizer.fit_transform(joined_description) # generate the tfidf vector representation for all articles
tfidf_features.shape

(776, 5168)

In [10]:
def write_vectorFile(data_features,filename):
    num = data_features.shape[0] # the number of document
    out_file = open(filename, 'w') # creates a txt file and open to save the vector representation
    for a_ind in range(0, num): # loop through each article by index
        for f_ind in data_features[a_ind].nonzero()[1]: # for each word index that has non-zero entry in the data_feature
            value = data_features[a_ind][0,f_ind] # retrieve the value of the entry from data_features
            out_file.write("{}:{} ".format(f_ind,value)) # write the entry to the file in the format of word_index:value
        out_file.write('\n') # start a new line after each article
    out_file.close() # close the file   

In [11]:
tfidf_features_file = "jobs_data_vector.txt" # file name of the tfidf vector

write_vectorFile(tfidf_features,tfidf_features_file) # write the tfidf vector to file

## Creating Model based on word embeddings

#### Creating and Saving FastText Model

In [12]:
from gensim.models.fasttext import FastText

# 1. Set the corpus file names/path
corpus_file = 'description.txt'

# 2. Initialise the Fast Text model
bbcFT = FastText(vector_size=50) 

# 3. build the vocabulary
bbcFT.build_vocab(corpus_file=corpus_file)

# 4. train the model
bbcFT.train(
    corpus_file=corpus_file, epochs=bbcFT.epochs,
    total_examples=bbcFT.corpus_count, total_words=bbcFT.corpus_total_words,
)

print(bbcFT)

FastText<vocab=2741, vector_size=50, alpha=0.025>


We can retrieve the KeyedVectors from the model as follows,

In [13]:
bbcFT_wv = bbcFT.wv
print(bbcFT_wv)

FastTextKeyedVectors<vector_size=50, 2741 keys>


#### Save the model
we also save our trained FastText model using the standard gensim methods. 

In [14]:
# Save the model
# bbcFT.save("models/FastText/bbcFT.model")

In [15]:
from gensim.models.fasttext import FastText
bbcFT = FastText.load("models/FastText/bbcFT.model")
print(bbcFT)
bbcFT_wv= bbcFT.wv

FastText<vocab=2741, vector_size=50, alpha=0.025>


we experiment the FastText embeddings. 
Similar, we:
* load the FastText model saved in our prevoius activity;
* generate document embeddings based on the load FastText word embeddings;
* explore the reprensentiveness of the features through tSNE;
* bulid the logistic regression model based on the generated document embeddings for news classfication.

In [16]:
def docvecs(embeddings, docs):
    vecs = np.zeros((len(docs), embeddings.vector_size))
    for i, doc in enumerate(docs):
        valid_keys = [term for term in doc if term in embeddings.key_to_index]
        docvec = np.vstack([embeddings[term] for term in valid_keys])
        """
        Note: using `sum` here, other 'pooling' options are possible too,
        e.g. mean, etc.
        """
        docvec = np.sum(docvec, axis=0)
        vecs[i,:] = docvec
    return vecs


def gen_docVecs(wv,tk_txts):
    docs_vectors = pd.DataFrame() # creating empty final dataframe
    #stopwords = nltk.corpus.stopwords.words('english') # if we haven't pre-processed the articles, it's a good idea to remove stop words

    for i in range(0,len(tk_txts)):
        tokens = tk_txts[i]
        temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
        for w_ind in range(0, len(tokens)): # looping through each word of a single document and spliting through space
            try:
                word = tokens[w_ind]
                word_vec = wv[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = pd.concat([temp, pd.Series(word_vec)], ignore_index = True)
            except:
                pass
        doc_vector = temp.sum() # take the sum of each column
        docs_vectors = pd.concat([docs_vectors, doc_vector], ignore_index = True)
    return docs_vectors

def plotTSNE(labels,features): # features as a numpy array, each element of the array is the document embedding of an article
    categories = sorted(labels.unique())
    # Sampling a subset of our dataset because t-SNE is computationally expensive
    SAMPLE_SIZE = int(len(features) * 0.3)
    np.random.seed(0)
    indices = np.random.choice(range(len(features)), size=SAMPLE_SIZE, replace=False)
    projected_features = TSNE(n_components=2, random_state=0).fit_transform(features[indices])
    colors = ['pink', 'green', 'midnightblue', 'orange', 'darkgrey']
    for i in range(0,len(categories)):
        points = projected_features[(labels[indices] == categories[i])]
        plt.scatter(points[:, 0], points[:, 1], s=30, c=colors[i], label=categories[i])
    plt.title("Feature vector for each article, projected on 2 dimensions.",
              fontdict=dict(fontsize=15))
    plt.legend()
    plt.show()

In [17]:
# NOTE this can take some time to finish running
# generate document embeddings
bbcFT_dvs = gen_docVecs(bbcFT_wv,jobs_data['Tokenized Description'])
bbcFT_dvs.isna().any().sum()

0

In [18]:
bbcFT_dvs = docvecs(bbcFT_wv, jobs_data['Tokenized Description'])

ValueError: need at least one array to concatenate

In [19]:
features = bbcFT_dvs.to_numpy()
plotTSNE(jobs_data['Category'],features)

ValueError: n_components=2 must be between 1 and min(n_samples, n_features)=1 with svd_solver='randomized'

### Building the weighted (i.e., TF-IDF weighted) and unweighted vector representation

#### Unweighted Document Vectors

### Saving outputs
Save the count vector representation as per spectification.
- count_vectors.txt

In [ ]:
# code to save output data...

## Task 3. Job Advertisement Classification

...... Sections and code blocks on buidling classification models based on different document feature represetations. 
Detailed comparsions and evaluations on different models to answer each question as per specification. 

<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [ ]:
# Code to perform the task...


## Summary
Give a short summary and anything you would like to talk about the assessment tasks here.

## Couple of notes for all code blocks in this notebook
- please provide proper comment on your code
- Please re-start and run all cells to make sure codes are runable and include your output in the submission.   
<span style="color: red"> This markdown block can be removed once the task is completed. </span>